In [46]:
# load data
import pickle
import numpy as np

with open("piano_X.pkl", "rb") as f:
    X = pickle.load(f)
    print('X:',len(X))

with open("piano_y.pkl", "rb") as f:
    y = pickle.load(f)
    print('X:',len(X))

X: 673098
X: 673098


In [47]:
import pandas as pd
y = pd.DataFrame(y, columns=['pitch','velocity'])
y = y['pitch'].astype(int)

In [48]:
y

0         57
1         81
2         62
3         86
4         64
          ..
673093    64
673094    67
673095    72
673096    40
673097    43
Name: pitch, Length: 673098, dtype: int32

In [10]:
# scale and split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
for i, note in enumerate(X):
    X[i] = scaler.fit_transform(note) # transform the whole thing



In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # split

In [50]:
# CNN classification model design

# import tensorflow as tf
# from tensorflow.keras import layers, models
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # Define the CNN model
# model = models.Sequential()

# # Input layer
# model.add(layers.Input(shape=(13, 50)))  # Assuming 50 time steps and 13 coefficients

# # Convolutional layers
# model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
# model.add(layers.MaxPooling1D(pool_size=2))

# # Flatten layer
# model.add(layers.Flatten())

# # Fully connected layers
# model.add(layers.Dense(units=256, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(units=128, activation='relu'))

# # Output layer
# num_pitch_classes = 87  # Adjust based on your pitch classes
# model.add(layers.Dense(units=num_pitch_classes, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Summary of the model architecture
# model.summary()

In [51]:
# import tensorflow as tf
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input
# from tensorflow.keras.models import Model


# input_shape = (13, 50)

# model = tf.keras.Sequential()
# model.add(Input(shape=input_shape))
# model.add(Conv1D(64, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(1, activation='linear'))  # Linear activation for regression
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_81 (Conv1D)          (None, 11, 64)            9664      
                                                                 
 max_pooling1d_77 (MaxPooli  (None, 5, 64)             0         
 ng1D)                                                           
                                                                 
 flatten_15 (Flatten)        (None, 320)               0         
                                                                 
 dense_45 (Dense)            (None, 128)               41088     
                                                                 
 dense_46 (Dense)            (None, 1)                 129       
                                                                 
Total params: 50881 (198.75 KB)
Trainable params: 50881 (198.75 KB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [53]:
model = tf.keras.Sequential()
model.add(Input(shape=input_shape))
model.add(tf.keras.layers.LSTM(64, activation='relu', return_sequences=True))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))  # Linear activation for regression
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 13, 64)            29440     
                                                                 
 flatten_16 (Flatten)        (None, 832)               0         
                                                                 
 dense_47 (Dense)            (None, 128)               106624    
                                                                 
 dense_48 (Dense)            (None, 1)                 129       
                                                                 
Total params: 136193 (532.00 KB)
Trainable params: 136193 (532.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
results = model.fit(X_train, y_train, 
                    epochs=15, batch_size=512,
                    validation_data=(X_test, y_test))

Epoch 1/15
1052/1052 [==============================] - 21s 19ms/step - loss: 180.0136 - mae: 10.2240 - val_loss: 140.4825 - val_mae: 9.3618
Epoch 2/15
1052/1052 [==============================] - 20s 19ms/step - loss: 137.1998 - mae: 9.2321 - val_loss: 132.4632 - val_mae: 9.1067
Epoch 3/15
1052/1052 [==============================] - 20s 19ms/step - loss: 128.6103 - mae: 8.9236 - val_loss: 129.9601 - val_mae: 8.8621
Epoch 4/15
1052/1052 [==============================] - 20s 19ms/step - loss: 124.5985 - mae: 8.7682 - val_loss: 123.5207 - val_mae: 8.7378
Epoch 5/15
1052/1052 [==============================] - 20s 19ms/step - loss: 121.6713 - mae: 8.6525 - val_loss: 124.8226 - val_mae: 8.6601
Epoch 6/15
1052/1052 [==============================] - 19s 18ms/step - loss: 120.1786 - mae: 8.5918 - val_loss: 119.2175 - val_mae: 8.5187
Epoch 7/15
1052/1052 [==============================] - 19s 18ms/step - loss: 117.5054 - mae: 8.4868 - val_loss: 120.7997 - val_mae: 8.5187
Epoch 8/15
1052/105

In [ ]:
# I keep getting mae close to 8. This is not ideal, even considered horrible, but that surprisingly is a whole octave